In [41]:
import pandas as pd
import numpy as np
import recordlinkage

In [42]:
innovaccer = pd.read_csv("Deduplication Problem - Sample Dataset.csv")
innovaccer.head()

,ln,dob,gn,fn
0,SMITH JR,01/03/68,F,WILLIAM
1,ROTHMEYER JR,01/03/68,F,WILLIAM
2,ASBY JR,01/03/68,F,WILLIAM
3,SALTER JR,01/03/68,F,WILLIAM
4,SALTER JR,01/03/68,F,WILLIAM


In [43]:
innovaccer.dtypes

ln     object
dob    object
gn     object
fn     object
dtype: object

In [44]:
innovaccer.describe()

,ln,dob,gn,fn
count,103,103,103,103
unique,28,22,2,26
top,MICHAELSON JR,07/10/37,M,HAROLD
freq,22,24,77,24


In [45]:
#to convert first name and last name to upper case
innovaccer['ln'] = innovaccer['ln'].str.upper()
innovaccer['fn'] = innovaccer['fn'].str.upper()
innovaccer.head()

,ln,dob,gn,fn
0,SMITH JR,01/03/68,F,WILLIAM
1,ROTHMEYER JR,01/03/68,F,WILLIAM
2,ASBY JR,01/03/68,F,WILLIAM
3,SALTER JR,01/03/68,F,WILLIAM
4,SALTER JR,01/03/68,F,WILLIAM


In [46]:
#to remove suffix in the last name
innovaccer['ln'] = innovaccer['ln'].str.replace("JR","")
innovaccer['ln'] = innovaccer['ln'].str.replace("SR","")
innovaccer['ln'] = innovaccer['ln'].str.replace("I","")
innovaccer['ln'] = innovaccer['ln'].str.replace("II","")
innovaccer['ln'] = innovaccer['ln'].str.replace("III","")
innovaccer.head()

,ln,dob,gn,fn
0,SMTH,01/03/68,F,WILLIAM
1,ROTHMEYER,01/03/68,F,WILLIAM
2,ASBY,01/03/68,F,WILLIAM
3,SALTER,01/03/68,F,WILLIAM
4,SALTER,01/03/68,F,WILLIAM


In [47]:
#str.strip() removes whitespaces from both left and right 
innovaccer['ln'] = innovaccer['ln'].str.strip()
innovaccer.head()

,ln,dob,gn,fn
0,SMTH,01/03/68,F,WILLIAM
1,ROTHMEYER,01/03/68,F,WILLIAM
2,ASBY,01/03/68,F,WILLIAM
3,SALTER,01/03/68,F,WILLIAM
4,SALTER,01/03/68,F,WILLIAM


In [48]:
#every whitespace is removed
innovaccer['fn'] = innovaccer['fn'].str.replace(" ","")
innovaccer.head()


,ln,dob,gn,fn
0,SMTH,01/03/68,F,WILLIAM
1,ROTHMEYER,01/03/68,F,WILLIAM
2,ASBY,01/03/68,F,WILLIAM
3,SALTER,01/03/68,F,WILLIAM
4,SALTER,01/03/68,F,WILLIAM


In [49]:
#to drop duplicates rows
dedupe = innovaccer.drop_duplicates()
dedupe.head()


,ln,dob,gn,fn
0,SMTH,01/03/68,F,WILLIAM
1,ROTHMEYER,01/03/68,F,WILLIAM
2,ASBY,01/03/68,F,WILLIAM
3,SALTER,01/03/68,F,WILLIAM
5,BLAND,21/02/62,F,WILLIAM


In [50]:
#full name = first name + last name
dedupe['full_name'] = dedupe[['fn', 'ln']].apply(lambda x: ''.join(x), axis=1)
dedupe.head()


C:\Users\Manohar Battula\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ln,dob,gn,fn,full_name
0,SMTH,01/03/68,F,WILLIAM,WILLIAMSMTH
1,ROTHMEYER,01/03/68,F,WILLIAM,WILLIAMROTHMEYER
2,ASBY,01/03/68,F,WILLIAM,WILLIAMASBY
3,SALTER,01/03/68,F,WILLIAM,WILLIAMSALTER
5,BLAND,21/02/62,F,WILLIAM,WILLIAMBLAND


In [51]:
#created pairs of the data frame
indexer = recordlinkage.FullIndex()
pairs = indexer.index(dedupe)
print (len(dedupe), len(pairs))

58 1653


In [52]:
#comparisons when date of birth is same
indexer = recordlinkage.BlockIndex(on='dob')
pairs = indexer.index(dedupe)
print (len(pairs))

89


In [53]:
#comparisions and binary computation
compare_cl = recordlinkage.Compare()
compare_cl.exact('dob', 'dob', label='dob')
compare_cl.string('full_name', 'full_name', method='jarowinkler', threshold=0.85, label='full_name')
compare_cl.exact('gn', 'gn', label='gn')
compare_cl.exact('ln', 'ln', label='ln')
features = compare_cl.compute(pairs, dedupe)
features.head()

dob  full_name  gn  ln
0 1    1        1.0   1   0
  2    1        1.0   1   0
  3    1        1.0   1   0
1 2    1        0.0   1   0
  3    1        0.0   1   0

In [54]:
features1 = features[(features['gn']==1) & (features['full_name']==1) & (features['ln']==1) & (features['dob']==1)]
features1 = features1.reset_index()
features1.head()

,level_0,level_1,dob,full_name,gn,ln
0,20,21,1,1.0,1,1
1,30,32,1,1.0,1,1
2,33,34,1,1.0,1,1
3,45,46,1,1.0,1,1
4,94,95,1,1.0,1,1


In [55]:
dedupe = dedupe.reset_index()
common = dedupe.merge(features1,left_on='index',right_on = 'level_1')
common.head(7)

,index,ln_x,dob_x,gn_x,fn,full_name_x,level_0,level_1,dob_y,full_name_y,gn_y,ln_y
0,21,CARLSON,25/10/53,F,RON,RONCARLSON,20,21,1,1.0,1,1
1,32,MCHAELSON,09/04/47,M,BOB,BOBMCHAELSON,30,32,1,1.0,1,1
2,34,MUSTAPHA,11/03/47,M,OWAN,OWANMUSTAPHA,33,34,1,1.0,1,1
3,46,CLARK,09/02/45,M,DONALD,DONALDCLARK,45,46,1,1.0,1,1
4,95,HANNA,24/11/34,M,ADDISONJOHN,ADDISONJOHNHANNA,94,95,1,1.0,1,1
5,96,HANNA,24/11/34,M,ADDISON,ADDISONHANNA,94,96,1,1.0,1,1
6,96,HANNA,24/11/34,M,ADDISON,ADDISONHANNA,95,96,1,1.0,1,1


In [56]:
dedupe = dedupe[(~dedupe.index.isin(common.index))]
dedupe.head()

,index,ln,dob,gn,fn,full_name
7,14,BLAND,08/06/54,F,WILLIAM,WILLIAMBLAND
8,16,BLAND,25/10/53,F,WILLIAM,WILLIAMBLAND
9,18,SHAFFER,25/10/53,F,WILLIAM,WILLIAMSHAFFER
10,19,DUNCAN,25/10/53,F,THOMAS,THOMASDUNCAN
11,20,CARLSON,25/10/53,F,ROY,ROYCARLSON


In [57]:
dedupe.reset_index()
len(dedupe)

51

In [58]:
dedupe.to_csv('Final output data.csv')